In [ ]:
"""
Made to extract saved data in files and plot it in graphs
"""

'\nMade to extract saved data in files\n'

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import mplhep as hep

In [8]:
def plot_hist_optimizer(optimizer_record, optimizer, topic_str, xlabel, save_location, showTitle=False):
    plt.hist(optimizer_record)
    plt.xlabel(xlabel)
    plt.ylabel("Frequency")
    if (showTitle):
        plt.title(topic_str + f" - Alternating Ansatz + KL divergence + {optimizer}")
    plt.savefig(save_location, bbox_inches='tight')
    plt.clf()

def plot_all_histogram(record, optimizers, topic_str, xlabel, save_location, showTitle=False):
    flat_record = [i for list in record for i in list]   
    bins = np.histogram(flat_record, bins=25)[1]

    plt.hist(record, bins=bins, alpha = 0.7, label = optimizers)


    plt.xlabel(xlabel)
    plt.ylabel("Frequecy")
    plt.legend()

    if(showTitle):
        plt.title("Main " + topic_str + f" - Alternating Ansatz + KL divergence")

    plt.savefig(save_location, bbox_inches='tight')
    plt.clf()

def plot_strip_plot(record, optimizers, topic_str, ylabel, save_location, includeBox=True, logScale=False, showTitle=False, invertYAxis=False):
    if (invertYAxis):
        plt.gca().invert_yaxis()
    
    reformatted_record = [(element, optimizers[i]) for i, optimizer_record in enumerate(record) for element in optimizer_record]
    df = pd.DataFrame(reformatted_record, columns=[ylabel, "Optimizers"])
    
    ax = sns.stripplot(data=df, x="Optimizers", y=ylabel, hue="Optimizers", legend=False)

    if (includeBox):
        sns.boxplot(data=df, x="Optimizers", y=ylabel, showcaps=False, boxprops={'facecolor':'None'},
                    showfliers=False,whiskerprops={'linewidth':0})
    
    if (logScale):
        plt.yscale('log')

    if(showTitle):
        plt.title("Main " + topic_str + f" Strip Plot - Alternating Ansatz + KL divergence")
        
    plt.xticks(rotation = 90)
    plt.savefig(save_location, bbox_inches='tight')
    plt.clf()

In [ ]:
hep.style.use(hep.style.CMS)

reruns = 25

folder = "saved_final_no_seed_ghz_state"
# best optimizers will be from [0:not_important]
not_important = 7
# first 0-5 do reasonably well, 6-9 have poorer performance
optimizers = ["Powell", "PYBADS", "Parameter_Shift","Parameter_Shift_Not_Fixed", "Finite_Difference", "COBYLA", "SPSA", "SPSA_Optimized", "Nelder-Mead", "L-BFGS-B", "COBYQA", "SLSQP"]

"""
folder = "saved_final_no_seed_spin_chain_state"
# best optimizers will be from [0:not_important]
not_important = 6
# first 0-5 do reasonably well, 6-9 have poorer performance
optimizers = ["Powell", "PYBADS", "Parameter_Shift", "Finite_Difference", "COBYLA", "SPSA", "SPSA_Optimized", "Nelder-Mead", "L-BFGS-B", "COBYQA", "SLSQP"]
"""
function_call_records = [[-1 for _ in range(reruns)] for _ in range(len(optimizers))]
fidelities_record = [[-1 for _ in range(reruns)] for _ in range(len(optimizers))]
elapsed_cpu = [[-1 for _ in range(reruns)] for _ in range(len(optimizers))]
elapsed_wall = [[-1 for _ in range(reruns)] for _ in range(len(optimizers))]

for j, optimizer in enumerate(optimizers):
    loss_value_records = [0] * reruns
    
    total_elapsed_cpu = 0
    total_elapsed_wall = 0

    for i in range(0, reruns):

        loss_values = np.load(f"{folder}/{optimizer}/loss_training_iter{i}.npy")
        loss_value_records[i] = loss_values

        elapsed_cpu[j][i] = np.load(f"{folder}/{optimizer}/elapsed_cpu_times_iter{i}.npy").item()
        elapsed_wall[j][i] = np.load(f"{folder}/{optimizer}/elapsed_wall_times_iter{i}.npy").item()
        
        total_elapsed_cpu += elapsed_cpu[j][i]
        total_elapsed_wall += elapsed_wall[j][i]

        fidelities_record[j][i] = np.load(f"{folder}/{optimizer}/fidelity_iter{i}.npy").item()

        if (optimizer[:15] != "Parameter_Shift"):
            function_call_records[j][i] = len(loss_values)
        else:
            function_call_records[j][i] = np.load(f"{folder}/{optimizer}/num_function_calls_iter{i}.npy").item()

inv_fidelities = [[(1 - fidelity) for fidelity in fidelities] for fidelities in fidelities_record]
        

<Figure size 1000x1000 with 0 Axes>

In [ ]:
### ALL OPTIMIZERS:
# plot all fidelities into histogram
plot_all_histogram(fidelities_record, optimizers, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_all.png")

# plot the number function calls of all optimizers as a histogram        
plot_all_histogram(function_call_records, optimizers, "Func. Calls", "Function Calls", f"{folder}/plots_main_func_call_all.png")

# plot all the wall times into histogram
plot_all_histogram(elapsed_wall, optimizers, "Wall Time", "Wall Time (sec)", f"{folder}/plots_main_wall_time_all.png")

# plot all the cpu times into histogram
plot_all_histogram(elapsed_cpu, optimizers, "CPU Time", "CPU Time (sec)", f"{folder}/plots_main_cpu_time_all.png")


## plotting strip plots with boxes
# plot strip plot of fidelities 
plot_strip_plot(fidelities_record, optimizers, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_stripplot_all.png")

# plot strip plot of fidelities zoomed in
plt.ylim(bottom = 0.95)
plot_strip_plot(fidelities_record, optimizers, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_zoom_stripplot_all.png")

# plot the number function calls of all optimizers as a strip plot        
plot_strip_plot(function_call_records, optimizers, "Func. Calls", "Function Calls", f"{folder}/plots_main_func_call_stripplot_all.png")

# plot strip plot of wall times
plot_strip_plot(elapsed_wall, optimizers, "Wall Time", "Wall Time (sec)", f"{folder}/plots_main_wall_time_stripplot_all.png")

# plot strip plot of cpu times 
plot_strip_plot(elapsed_cpu, optimizers, "CPU Time", "CPU Time (sec)", f"{folder}/plots_main_cpu_time_stripplot_all.png")

# plot strip plot of inverse fidelity 1-fidelity
plot_strip_plot(inv_fidelities, optimizers, "Inverse Fidelity", "Inverse Fidelity", f"{folder}/plots_main_inv_fidelity_stripplot_all.png", logScale=True, invertYAxis=True)


## now plotting without boxes in strip plot
# plot strip plot of fidelities 
plot_strip_plot(fidelities_record, optimizers, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_stripplot2_all.png", False)

plt.ylim(bottom = 0.95)
plot_strip_plot(fidelities_record, optimizers, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_zoom_stripplot2_all.png", False)

# plot the number function calls of all optimizers as a strip plot          
plot_strip_plot(function_call_records, optimizers, "Func. Calls", "Function Calls", f"{folder}/plots_main_func_call_stripplot2_all.png", False)

# plot strip plot of wall times
plot_strip_plot(elapsed_wall, optimizers, "Wall Time", "Wall Time (sec)", f"{folder}/plots_main_wall_time_stripplot2_all.png", False)

# plot strip plot of cpu times 
plot_strip_plot(elapsed_cpu, optimizers, "CPU Time", "CPU Time (sec)", f"{folder}/plots_main_cpu_time_stripplot2_all.png", False)

# plot strip plot of inverse fidelity 1-fidelity
plot_strip_plot(inv_fidelities, optimizers, "Inverse Fidelity", "Inverse Fidelity", f"{folder}/plots_main_inv_fidelity_stripplot2_all.png", includeBox=False, logScale=True, invertYAxis=True)



### JUST THE GOOD OPTIMIZERS:
optimizers_best = optimizers[:not_important]
fidelities_record_best = fidelities_record[:not_important]
inv_fidelities_best = inv_fidelities[:not_important]
function_call_records_best = function_call_records[:not_important]
elapsed_wall_best = elapsed_wall[:not_important]
elapsed_cpu_best = elapsed_cpu[:not_important]

# plot all fidelities into histogram
plot_all_histogram(fidelities_record_best, optimizers_best, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_best.png")

# plot the number function calls of all optimizers as a histogram        
plot_all_histogram(function_call_records_best, optimizers_best, "Func. Calls", "Function Calls", f"{folder}/plots_main_func_call_best.png")

# plot all the wall times into histogram
plot_all_histogram(elapsed_wall_best, optimizers_best, "Wall Time", "Wall Time (sec)", f"{folder}/plots_main_wall_time_best.png")

# plot all the cpu times into histogram
plot_all_histogram(elapsed_cpu_best, optimizers_best, "CPU Time", "CPU Time (sec)", f"{folder}/plots_main_cpu_time_best.png")


## plotting strip plots with boxes
# plot strip plot of fidelities 
plot_strip_plot(fidelities_record_best, optimizers_best, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_stripplot_best.png")

# plot strip plot of fidelities zoomed in
plt.ylim(bottom = 0.95)
plot_strip_plot(fidelities_record_best, optimizers_best, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_zoom_stripplot_best.png")

# plot the number function calls of all optimizers as a strip plot        
plot_strip_plot(function_call_records_best, optimizers_best, "Func. Calls", "Function Calls", f"{folder}/plots_main_func_call_stripplot_best.png")

# plot strip plot of wall times
plot_strip_plot(elapsed_wall_best, optimizers_best, "Wall Time", "Wall Time (sec)", f"{folder}/plots_main_wall_time_stripplot_best.png")

# plot strip plot of cpu times 
plot_strip_plot(elapsed_cpu_best, optimizers_best, "CPU Time", "CPU Time (sec)", f"{folder}/plots_main_cpu_time_stripplot_best.png")

# plot strip plot of inverse fidelity 1-fidelity
plot_strip_plot(inv_fidelities_best, optimizers, "Inverse Fidelity", "Inverse Fidelity", f"{folder}/plots_main_inv_fidelity_stripplot_best.png", logScale=True, invertYAxis=True)


## now plotting without boxes in strip plot
# plot strip plot of fidelities 
plot_strip_plot(fidelities_record_best, optimizers_best, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_stripplot2_best.png", False)

plt.ylim(bottom = 0.95)
plot_strip_plot(fidelities_record_best, optimizers_best, "Fidelity", "Fidelity", f"{folder}/plots_main_fidelity_zoom_stripplot2_best.png", False)

# plot the number function calls of all optimizers as a strip plot          
plot_strip_plot(function_call_records_best, optimizers_best, "Func. Calls", "Function Calls", f"{folder}/plots_main_func_call_stripplot2_best.png", False)

# plot strip plot of wall times
plot_strip_plot(elapsed_wall_best, optimizers_best, "Wall Time", "Wall Time (sec)", f"{folder}/plots_main_wall_time_stripplot2_best.png", False)

# plot strip plot of cpu times 
plot_strip_plot(elapsed_cpu_best, optimizers_best, "CPU Time", "CPU Time (sec)", f"{folder}/plots_main_cpu_time_stripplot2_best.png", False)

# plot strip plot of inverse fidelity 1-fidelity
plot_strip_plot(inv_fidelities_best, optimizers, "Inverse Fidelity", "Inverse Fidelity", f"{folder}/plots_main_inv_fidelity_stripplot2_best.png", includeBox=False, logScale=True, invertYAxis=True)